In [1]:
import numpy as np
import pandas as pd
%matplotlib notebook

In [2]:
n = str(2)
fullDF = pd.read_csv("data/Chiller"+n+"_full.csv")
num_samples = fullDF.shape[0]
window_size = 60

In [3]:
fullDF.head()

,ts,ch1Watt,ch2Watt,ch3Watt,value1,value2,value3,value4,conflowRate,conflowSpeed,evaflowRate,evaflowSpeed
0,2017-05-01 00:00:00,325,0,324,18.580934,21.582337,23.633118,23.451539,0.0,0.0,0.0,0.0
1,2017-05-01 00:01:00,326,0,328,18.602295,21.539612,23.558351,23.398133,0.0,0.0,0.0,0.0
2,2017-05-01 00:02:00,327,0,324,18.495484,21.614379,23.483581,23.323363,0.0,0.0,0.0,0.0
3,2017-05-01 00:03:00,338,0,325,18.474123,21.571657,23.601076,23.205871,0.0,0.0,0.0,0.0
4,2017-05-01 00:04:00,334,0,328,18.559570,21.560973,23.536987,23.227236,0.0,0.0,0.0,0.0


In [4]:
Watt = fullDF[["ch1Watt", "ch2Watt", "ch3Watt"]].as_matrix()
Value = fullDF[["value1", "value2", "value3", "value4"]].as_matrix()
Conflow = fullDF[["conflowRate", "conflowSpeed"]].as_matrix()
Evaflow = fullDF[["evaflowRate", "evaflowSpeed"]].as_matrix()
Total_Watt = np.sum(Watt, axis = 1)

In [5]:
from sklearn import preprocessing

In [6]:
Watt_scaled = preprocessing.scale(Watt)
Value_scaled = preprocessing.scale(Value)
Conflow_scaled = preprocessing.scale(Conflow)
Evaflow_scaled = preprocessing.scale(Evaflow)
Total_Watt_scaled = preprocessing.scale(Total_Watt)

/Users/duynguyen/anaconda/envs/python2/lib/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


In [7]:
Hour = pd.to_datetime(fullDF.ts).dt.hour.as_matrix()
Hour_transformed = np.zeros([num_samples,24])
for i in range(num_samples):
    Hour_transformed[i, Hour[i]] = 1

In [8]:
Day = pd.to_datetime(fullDF.ts).dt.day.as_matrix()
Day_transformed = np.zeros([num_samples,31])
for i in range(num_samples):
    Day_transformed[i, Day[i] - 1] = 1

In [9]:
Month = pd.to_datetime(fullDF.ts).dt.month.as_matrix()
Month_transformed = np.zeros([num_samples,12])
for i in range(num_samples):
    Month_transformed[i, Month[i] - 1] = 1

In [160]:
Data = np.concatenate([Hour_transformed, Value_scaled, Conflow_scaled, Evaflow_scaled], axis =1)
# Data = np.concatenate([Value_scaled, Conflow_scaled, Evaflow_scaled], axis =1)

In [161]:
dimension = Data.shape[1]

In [162]:
Data_align = np.zeros((window_size, num_samples - window_size + 1, dimension))
for i in range(0, window_size):
    Data_align[i] = Data[i: num_samples - window_size + 1 + i]

In [163]:
Full_Data = Data_align.transpose((1,0,2))

In [182]:
Power = Total_Watt[:num_samples - window_size + 1].reshape((-1,1))

In [183]:
import keras
from keras import layers
from keras.layers import Lambda, Activation,recurrent, Bidirectional, Dense, Flatten, Conv1D, Dropout, LSTM, GRU, concatenate, multiply, add, Reshape, MaxPooling1D, BatchNormalization
from keras.models import Model, load_model
import keras.backend as K

In [184]:
def mean_pred(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true)/y_true)

In [185]:
inp = layers.Input(shape=(window_size, dimension), dtype=Full_Data.dtype)

In [186]:
flatten = Flatten()(inp)
dense1 = Dense(500, activation='relu')(flatten)
dense2 = Dense(500, activation='relu')(dense1)
dense3 = Dense(500, activation='relu')(dense2)

In [187]:
output = Dense(1)(dense3)

In [188]:
model = Model(inp,output)
model.compile(optimizer='rmsprop',
              loss='mean_absolute_error',
              metrics=[mean_pred])

In [189]:
test_inp = Full_Data[:40000]
test_out = Power[:40000]
val_inp = Full_Data[40000:45000]
val_out = Power[40000:45000]

In [ ]:
model.fit(test_inp, test_out,
          epochs=5,
          validation_data=(val_inp, val_out))

Train on 40000 samples, validate on 5000 samples
Epoch 1/5
10528/40000 [======>.......................] - ETA: 22s - loss: 8959.0550 - mean_pred: 0.7020

In [138]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 60, 75)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4500)              0         
_________________________________________________________________
dense_24 (Dense)             (None, 500)               2250500   
_________________________________________________________________
dense_25 (Dense)             (None, 500)               250500    
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 501       
Total params: 2,501,501
Trainable params: 2,501,501
Non-trainable params: 0
_________________________________________________________________


In [178]:
pred = model.predict(val_inp, verbose=1)

4928/5000 [============================>.] - ETA: 0s

In [179]:
pred

array([[  99378.015625 ],
       [ 100172.140625 ],
       [ 100299.1796875],
       ..., 
       [ 151416.640625 ],
       [ 152316.1875   ],
       [ 152783.640625 ]], dtype=float32)

In [180]:
(pred - val_out)/val_out

array([[-0.01507433, -0.01093767, -0.04023395, ..., -0.35679741,
        -0.35352119, -0.35421855],
       [-0.00720383, -0.00303412, -0.03256451, ..., -0.35165761,
        -0.34835521, -0.34905814],
       [-0.00594476, -0.00176976, -0.0313376 , ..., -0.35083538,
        -0.34752879, -0.34823261],
       ..., 
       [ 0.50067533,  0.50697812,  0.46234104, ..., -0.01998873,
        -0.01499694, -0.01605947],
       [ 0.50959066,  0.51593088,  0.47102862, ..., -0.01416661,
        -0.00914516, -0.010214  ],
       [ 0.51422354,  0.52058322,  0.47554316, ..., -0.01114112,
        -0.00610426, -0.00717638]])

In [181]:
val_out

array([100899, 100477, 103544, ..., 154505, 153722, 153888])